# Human Detection Project

In [ ]:
import numpy as np
import cv2
import os
import random
from skimage.feature import hog
from skimage import exposure
from sklearn import svm
from sklearn.decomposition import PCA
from skimage.transform import pyramid_gaussian
import matplotlib.pyplot as plt

In [ ]:
def showImg(im,text = 'xd'):
    cv2.imshow(text,im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Lectura de imágenes

In [ ]:
def leerFotos(path):
    os.chdir(path)
    listaPos = os.listdir()
    fotosDic = {}
    for i in range(len(listaPos)):
        fotosDic[i] = cv2.imread(listaPos[i],0)
    return fotosDic

### Fotos Tain 

In [ ]:
def duplicaTrani(fotos):
    indx = len(fotos)
    for i in range(len(fotos)):
        fotos[indx+i] = cv2.flip(fotos[i], 1)
        
    return fotos

In [ ]:
path = 'C:\\Users\\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\train_pos'
fotosPosTrain0 = leerFotos(path)
fotosPosTrain0 = duplicaTrani(fotosPosTrain0)

path = 'C:\\Users\\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\train_neg'
fotosNegTrainSinTratar = leerFotos(path)

### Fotos Test

In [ ]:
path = 'C:\\Users\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\Test\\pos'
fotosTest0 = leerFotos(path)

### Recorte Train Neg

In [ ]:
def generaImagenesRecortadas(winSize,dataNeg,numeroCortesxPh = 20):
    rowsFotoPersona = winSize[0]
    colsFotoPersona = winSize[1]
    
    fotosNegTrain = {}
    for i in range(len(dataNeg)):      
        rowsFotoEntorno = dataNeg[i].shape[0]
        colsFotoEntorno = dataNeg[i].shape[1]
        dimCol = colsFotoEntorno - colsFotoPersona
        dimRow = rowsFotoEntorno - rowsFotoPersona
        
        for j in range(numeroCortesxPh):    
            randRow = random.randint(0,dimRow-1)
            randCol = random.randint(0,dimCol-1)
            randomImage = dataNeg[i][randRow:randRow + rowsFotoPersona ,randCol: randCol + colsFotoPersona]
            fotosNegTrain[j+numeroCortesxPh*i] = randomImage
            
    return fotosNegTrain

In [ ]:
winSize = fotosPosTrain0[0].shape
fotosNegTrain0 = generaImagenesRecortadas(winSize,fotosNegTrainSinTratar)

# Tratamiento de la imagenes Train

### Ampliazcion Rango Dinámico && Suavizado && Normlizacion

In [ ]:
def ampRangoSet(fotos):
    fotSal = {}
    for i in range(len(fotos)):
        r1 = np.min(fotos[i])
        r2 = np.max(fotos[i])
        fotSal[i] = np.float32(fotos[i])
        fotSal[i] = 255*(fotSal[i]-r1)/(r2-r1)
        fotSal[i] = np.uint8(fotSal[i])
        
    return fotSal

In [ ]:
def trataFotos(fotos,kernelSize = 3):
    fotSal = {}
    ## Ampliación del rango
    fotSal = ampRangoSet(fotos)

    ## Suavizado
    #kernel = np.ones((kernelSize,kernelSize))/(kernelSize**2)
    n = kernelSize #tamaño del filtro
    sigma = 3 #desviación de la gaussiana
    mask = cv2.getGaussianKernel(n, sigma)*cv2.getGaussianKernel(n, sigma).T
    
    for i in range(len(fotSal)):
        fotSal[i] = cv2.filter2D(fotSal[i],-1,mask)
        
    return fotSal

In [ ]:
dimFiltro = 7
fotosNegTrain = trataFotos(fotosNegTrain0,dimFiltro)
fotosPosTrain = trataFotos(fotosPosTrain0,dimFiltro)
fotosTest = trataFotos(fotosTest0,dimFiltro)

fotosNegTrain[0].shape

# Extracción de Caracterísitcas

## Hog
    L1
       Normalization using L1-norm.
    L1-sqrt
       Normalization using L1-norm, followed by square root.
    L2
       Normalization using L2-norm.
    L2-Hys
       Normalization using L2-norm, followed by limiting the
       maximum values to 0.2 (`Hys` stands for `hysteresis`) and
       renormalization using L2-norm. (default)

In [ ]:
orientations    = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)
block_norm      = 'L2' 
visualize       = False
visualise       = None
transform_sqrt  = True
feature_vector  = True

In [ ]:
def featuresExtrac(imagenes,esPositivo):    
    ## Tamaño del vector
    auxDim = hog(imagenes[0],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector)
    tamHog = len(auxDim)
    
    #inicializamos la matriz que guarda los imagenes vectolizados
    fdImagenesHog = np.zeros((len(imagenes),tamHog))
    
    for i in range(len(imagenes)):
        fdImagenesHog[i,:] = hog(imagenes[i],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector)
        
    #asignamos el vector de clases(y) 
    if esPositivo:
        y = np.ones((len(imagenes),1))
    else:
        y = np.zeros((len(imagenes),1))

    return fdImagenesHog,y

In [ ]:
fdPosTrainHog,yPos = featuresExtrac(fotosPosTrain,1)

In [ ]:
fdNegTrainHog,yNeg = featuresExtrac(fotosNegTrain,0)

In [ ]:
fdTrainHog = np.vstack((fdPosTrainHog,fdNegTrainHog))
y = np.vstack((yPos,yNeg))


# Entrenamiento del modelo

## HOG Model

In [ ]:
clfHog = svm.LinearSVC(C=0.01)
clfHog.fit(fdTrainHog[:,:],y[:,:].ravel())

In [ ]:
clfHog.score(fdPosTrainHog,yPos)

In [ ]:
clfHog.score(fdNegTrainHog,yNeg)

# Slide Window

In [ ]:
def slidWindow(imagen,maskHeight = 134,maskWidth = 70,step = 10):
    arrayImagen = []
    
    for i in range(0,imagen.shape[0]-maskHeight,step):        
        for j in range(0,imagen.shape[1]-maskWidth,step):
            arrayImagen.append((i,j,imagen[i:i+maskHeight, j:j+maskWidth]))
    
    return arrayImagen

# Pyramid Gaussian

In [ ]:
def estudiaFoto(im,winSize,clfHog,ratio = 2,margen = 0):#Ratio de reducción
    # List to store the detections
    hdetections = []
    detectJump = 0;
    
    for scaLev,imScaled in enumerate(pyramid_gaussian(im,downscale=ratio,multichannel=False)):
        if (imScaled.shape[0] >= winSize[0] and imScaled.shape[1] >= winSize[1]):
            for (x,y,imVentana) in slidWindow(imScaled):
                if (detectJump <= 0):
                    fdHog = hog(imVentana,orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector);
                    predHog = clfHog.predict(fdHog.reshape(1,-1))
                    acc = clfHog.decision_function(fdHog.reshape(1,-1))[0]
                    
                    if (predHog == 1 and acc > margen):
                        detectJump = 10
                        hdetections.append((x*(ratio**scaLev),y*(ratio**scaLev),int(winSize[1]*(ratio**scaLev)),int(winSize[0]*(ratio**scaLev))))
                detectJump -= 1;
    return hdetections

# Prueba del Modelo

In [ ]:
def muestraResultado(detections,im):
    color = (255)
    for (x, y,w, h) in detections:
            # Draw the detections
            cv2.rectangle(im, (y, x), (y+w, x+h), color, thickness=1)
    cv2.imshow("Deteciones", im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
%%time
humanDetec = estudiaFoto(fotosTest[0],winSize,clfHog,ratio=2)

In [ ]:
len(humanDetec)

In [ ]:
muestraResultado(humanDetec,fotosTest[0].copy())

#### Prueba solo Una imagen

In [ ]:
%%time
listOk = []
partImagen = slidWindow(fotosTest[7])
lim = 0.5
for i in range(len(partImagen)):
    pruebaHog = hog(partImagen[i][2],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector);
    precis = clfHog.decision_function(pruebaHog.reshape(1,-1))[0]
    if (1 == clfHog.predict(pruebaHog.reshape(1,-1)) and precis > lim):
        listOk.append(i);
        showImg(partImagen[i][2])

In [ ]:
print(len(partImagen))
print(len(listOk))
print(listOk)

In [ ]:
cv2.imshow('img ', fotosTest[0])
cv2.imshow('trozo test ', partImagen[3520][2])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
pruebaHog = hog(partImagen[3520][2],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector);
porce = clfHog.decision_function(pruebaHog.reshape(1,-1))[0]
print(porce)